In [32]:
import numpy as np
import pandas as pd
import os

In [33]:
# diretorio dos dados
root = 'bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_csv'

# todos os arquivos no diretório
paths = os.listdir(root)
paths.sort() # ordena as pastas

# filtra os sinais
path_signals = [os.path.join(root, path) for path in paths if 'Signals' in path]

print(f'Primeiro sinal: {path_signals[0]}')
print(f'Último sinal:   {path_signals[-1]}')

Primeiro sinal: bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_csv/bidmc_01_Signals.csv
Último sinal:   bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_csv/bidmc_53_Signals.csv


In [34]:
def create_windows(arr, size_win, hold=0):
    windows = list()  # Lista para armazenar as janelas geradas

    # Percorre o array com um passo de (tamanho da janela - hold)
    for w in range(0, arr.shape[0], size_win-hold):
        win = arr[w:w+size_win]  # cria uma janela de tamanho size_win

        # Verifica se a janela tem o tamanho exato
        if len(win) == size_win:  
            # Adiciona a janela à lista
            windows.append(win)  
    
    return np.array(windows)

In [35]:
arr = np.array(range(1, 11))

print('Dataset com janelamento (sem sobreposição):')

print(create_windows(arr, 5))
print('Dataset com janelamento (com sobreposição):')
print(create_windows(arr, 5, 4))

Dataset com janelamento (sem sobreposição):
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]]
Dataset com janelamento (com sobreposição):
[[ 1  2  3  4  5]
 [ 2  3  4  5  6]
 [ 3  4  5  6  7]
 [ 4  5  6  7  8]
 [ 5  6  7  8  9]
 [ 6  7  8  9 10]]


In [42]:
def create_dataset_signal(path_signals, index, size_win=0, hold_size = 0, type_signal=' PLETH'):
    # carrega o csv
    df_signal = pd.read_csv(path_signals[index])
    df_signal['ECG_PPG'] = df_signal.apply(lambda row: [row[' PLETH'], row[' II']], axis=1)
    # filtra a caracteristica utilizada
    if size_win > 0:
        values_arr = df_signal[type_signal].values #array dos valores

        # cria um dataframe em que as linhas são uma janela temporal
        windows = create_windows(values_arr, size_win, hold_size)
        df_signal = pd.DataFrame(windows)

    else:
        # sem janelamento
        df_signal = df_signal[[' PLETH']].T

    df_signal['label'] = index + 1 # label
        
    return df_signal

In [44]:
def create_dataset(path_signals, size_win=80, hold_size=0, type_signal=' PLETH'):
    n_register = len(path_signals) # numero de dados
    df = pd.DataFrame() # armazena todos os dasets

    for i in range(n_register):
        # cria o dataset de cada usuario
        df1 = create_dataset_signal(path_signals, index=i, size_win=size_win, 
                                    hold_size=hold_size, type_signal=type_signal)

        # concatena todos os datasets
        df = pd.concat((df, df1), axis=0)

    return df

create_dataset(path_signals, type_signal='ECG_PPG')

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,label
0,"[0.43597, 0.72549]","[0.43206, 0.67059]","[0.42815, 0.6098]","[0.42424, 0.55098]","[0.42131, 0.5]","[0.41838, 0.4549]","[0.4174, 0.4098]","[0.41642, 0.3902]","[0.41544, 0.37451]","[0.41447, 0.35882]",...,"[0.48094, 0.51569]","[0.47801, 0.55098]","[0.47507, 0.60588]","[0.47116, 0.6549]","[0.46823, 0.69608]","[0.46334, 0.72549]","[0.45846, 0.74118]","[0.45357, 0.75098]","[0.44868, 0.74118]",1
1,"[0.44379, 0.71569]","[0.43891, 0.66471]","[0.435, 0.60588]","[0.43109, 0.5451]","[0.4262, 0.47451]","[0.42326, 0.42941]","[0.42033, 0.38431]","[0.41838, 0.3451]","[0.41642, 0.31961]","[0.41447, 0.3098]",...,"[0.4477, 0.57059]","[0.44379, 0.6098]","[0.43891, 0.67647]","[0.43304, 0.73137]","[0.43011, 0.77647]","[0.42717, 0.8]","[0.42522, 0.8]","[0.42229, 0.8]","[0.41935, 0.77647]",1
2,"[0.4174, 0.71569]","[0.41544, 0.65098]","[0.41349, 0.58431]","[0.41251, 0.50588]","[0.41153, 0.43529]","[0.41153, 0.37059]","[0.41153, 0.32941]","[0.41251, 0.2902]","[0.41349, 0.25882]","[0.41349, 0.23922]",...,"[0.44086, 0.52549]","[0.43695, 0.5902]","[0.43206, 0.61961]","[0.42815, 0.67647]","[0.42522, 0.72157]","[0.42326, 0.75098]","[0.42131, 0.77647]","[0.41935, 0.77059]","[0.4174, 0.77059]",1
3,"[0.41544, 0.7451]","[0.41349, 0.6902]","[0.41251, 0.62941]","[0.41056, 0.56471]","[0.40958, 0.50588]","[0.4086, 0.4549]","[0.40762, 0.41569]","[0.40762, 0.3902]","[0.40762, 0.36471]","[0.40762, 0.3451]",...,"[0.47214, 0.51961]","[0.46823, 0.56471]","[0.4653, 0.6]","[0.46237, 0.6451]","[0.46041, 0.69608]","[0.45846, 0.7451]","[0.4565, 0.77647]","[0.45357, 0.79608]","[0.44966, 0.79608]",1
4,"[0.44575, 0.78039]","[0.44184, 0.74118]","[0.43793, 0.7]","[0.43304, 0.65098]","[0.42913, 0.58431]","[0.42424, 0.52549]","[0.41838, 0.48039]","[0.41447, 0.4549]","[0.41153, 0.42941]","[0.40958, 0.41569]",...,"[0.45552, 0.48039]","[0.45259, 0.52941]","[0.44966, 0.57059]","[0.44575, 0.61961]","[0.44086, 0.68627]","[0.435, 0.72549]","[0.43011, 0.74118]","[0.42522, 0.7451]","[0.42131, 0.7549]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,"[2.1466, 0.2346]","[2.1848, 0.43988]","[2.2141, 0.71505]","[2.2346, 0.68524]","[2.2444, 0.73021]","[2.2444, 0.50978]","[2.2356, 0.31965]","[2.217, 0.3001]","[2.1906, 0.3651]","[2.1554, 0.41007]",...,"[1.2395, 0.3001]","[1.3412, 0.28983]","[1.4536, 0.28495]","[1.5953, 0.29472]","[1.7537, 0.30499]","[1.9189, 0.3001]","[2.0821, 0.28495]","[2.2366, 0.26491]","[2.3763, 0.21457]",53
746,"[2.5015, 0.20968]","[2.61, 0.28006]","[2.7038, 0.3348]","[2.7849, 0.37488]","[2.8426, 0.37977]","[2.8856, 0.39003]","[2.9277, 0.37977]","[2.9658, 0.36999]","[2.9971, 0.35973]","[3.0205, 0.35484]",...,"[0.99218, 0.28495]","[0.97849, 0.26979]","[0.96676, 0.23998]","[0.95601, 0.20968]","[0.94624, 0.21457]","[0.93744, 0.24976]","[0.9306, 0.29472]","[0.92375, 0.3348]","[0.92082, 0.3651]",53
747,"[0.91984, 0.37488]","[0.91691, 0.36999]","[0.913, 0.37488]","[0.90811, 0.3651]","[0.90323, 0.35484]","[0.89834, 0.35484]","[0.89638, 0.33969]","[0.89932, 0.32991]","[0.91202, 0.31476]","[0.93842, 0.31476]",...,"[1.2278, 0.2346]","[1.2053, 0.22972]","[1.1838, 0.21994]","[1.1681, 0.21994]","[1.1554, 0.2346]","[1.1398, 0.2346]","[1.1222, 0.20968]","[1.1036, 0.19501]","[1.085, 0.21457]",53
748,"[1.0674, 0.24976]","[1.0499, 0.28006]","[1.0332, 0.3001]","[1.0186, 0.32991]","[1.0039, 0.34995]","[0.99022, 0.35484]","[0.97654, 0.35973]","[0.96774, 0.3348]","[0.96188, 0.31476]","[0.95308, 0.30987]",...,"[1.1916, 0.26491]","[1.173, 0.29472]","[1.1769, 0.31476]","[1.1935, 0.32502]","[1.1935, 0.32991]","[1.1867, 0.30987]","[1.1769, 0.3001]","[1.1662, 0.28983]","[1.1554, 0.28495]",53


In [16]:
df_ecg = create_dataset(path_signals, size_win=0, type_signal=' PLETH')
df_ppg = create_dataset(path_signals, size_win=0, type_signal=' II')

# save os de ppg e ecg
df_ecg.to_csv('ecg.csv', index=False)
df_ppg.to_csv('ppg.csv', index=False)

In [24]:
pd.concat((df_ppg, df_ecg))

,0,1,2,3,4,5,6,7,8,9,...,59992,59993,59994,59995,59996,59997,59998,59999,60000,label
PLETH,0.43597,0.43206,0.42815,0.42424,0.42131,0.41838,0.41740,0.41642,0.41544,0.41447,...,0.41056,0.40958,0.40860,0.40762,0.40567,0.40371,0.40176,0.40176,0.40274,1
PLETH,0.63636,0.65689,0.67351,0.68817,0.69990,0.70870,0.71652,0.72043,0.72336,0.72434,...,0.58651,0.60313,0.61681,0.62854,0.63832,0.64516,0.65005,0.65298,0.65396,2
PLETH,0.18866,0.20137,0.21994,0.25220,0.29717,0.35288,0.41740,0.48778,0.55523,0.61290,...,0.29521,0.29814,0.30596,0.32063,0.34311,0.37341,0.41153,0.45552,0.49658,3
PLETH,0.67253,0.69892,0.72336,0.74291,0.75758,0.76833,0.77419,0.77517,0.77126,0.76442,...,0.29032,0.29228,0.29521,0.29717,0.29814,0.29912,0.30108,0.30205,0.30205,4
PLETH,0.41642,0.41153,0.40665,0.40078,0.39492,0.38905,0.38416,0.37928,0.37537,0.37243,...,0.33040,0.32551,0.32160,0.31965,0.31574,0.31183,0.30890,0.30596,0.30499,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PLETH,0.30401,0.29912,0.29619,0.29326,0.29130,0.29032,0.29032,0.29130,0.29228,0.29326,...,0.29326,0.29521,0.30108,0.31183,0.32551,0.34213,0.36657,0.39687,0.43011,49
PLETH,0.56500,0.56891,0.57185,0.57380,0.57380,0.57380,0.57380,0.57185,0.56891,0.56696,...,0.49853,0.49365,0.48876,0.48387,0.47801,0.47410,0.47019,0.46432,0.46041,50
PLETH,0.59531,0.57967,0.56403,0.54839,0.53568,0.52395,0.51222,0.50147,0.49071,0.48094,...,0.39492,0.38905,0.38416,0.37928,0.37537,0.36950,0.36461,0.35973,0.35582,51
PLETH,1.18480,1.16420,1.14660,1.13490,1.13390,1.15150,1.18280,1.23360,1.31570,1.42620,...,2.04690,1.95800,1.87590,1.80160,1.73700,1.68330,1.63930,1.60510,1.58060,52


In [30]:
import pandas as pd

df_ECG = pd.read_csv('ECG_test.csv')

In [31]:
df_ECG

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,label
0,0.34506,0.33920,0.33333,0.32747,0.32160,0.31574,0.30987,0.30499,0.29912,0.29326,...,0.40371,0.39980,0.39687,0.39394,0.39101,0.38612,0.38123,0.37634,0.37048,7.0
1,0.38221,0.37439,0.36755,0.36168,0.35484,0.34800,0.34115,0.33236,0.31672,0.29814,...,0.50635,0.48485,0.46432,0.44477,0.42717,0.41251,0.40078,0.39101,0.38221,37.0
2,2.74100,2.75560,2.76740,2.78200,2.79770,2.81040,2.82110,2.82600,2.82500,2.81620,...,1.05380,1.04300,1.03230,1.02150,1.01660,1.01370,1.00680,0.99804,0.99022,45.0
3,0.51906,0.51417,0.50929,0.50440,0.49951,0.49560,0.49071,0.48778,0.48289,0.47898,...,0.61681,0.61290,0.60802,0.60117,0.59433,0.58749,0.57967,0.56989,0.55914,12.0
4,0.49756,0.52590,0.55230,0.57674,0.59824,0.61193,0.61975,0.62659,0.63148,0.63636,...,0.62854,0.62952,0.62952,0.62659,0.62366,0.61779,0.61095,0.60313,0.59433,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31795,1.74580,1.88270,2.01080,2.12810,2.23260,2.32260,2.39880,2.45360,2.49360,2.52880,...,2.31570,2.43500,2.54450,2.64220,2.72530,2.79470,2.85040,2.89350,2.92470,46.0
31796,0.42913,0.43988,0.45650,0.47703,0.49756,0.52004,0.54643,0.57283,0.59824,0.62170,...,0.42424,0.42229,0.41838,0.41447,0.41056,0.40665,0.40176,0.39785,0.39492,37.0
31797,0.36364,0.35973,0.35582,0.35288,0.34897,0.34604,0.34213,0.34018,0.33822,0.33627,...,0.49169,0.47605,0.46237,0.45259,0.44477,0.43891,0.43402,0.43011,0.42815,9.0
31798,0.43891,0.43793,0.43793,0.43695,0.43597,0.43597,0.43597,0.43597,0.43597,0.43597,...,0.51613,0.51515,0.51417,0.51320,0.51026,0.50733,0.50440,0.50049,0.49658,50.0


In [25]:
import pandas as pd

# Exemplo de DataFrame
df = pd.DataFrame({
    'col1': ['a', 'b', 'c'],
    'col2': ['x', 'y', 'z']
})

# Usando apply para juntar as colunas em uma lista
df['unido'] = df.apply(lambda row: [row['col1'], row['col2']], axis=1)

print(df)


  col1 col2   unido
0    a    x  [a, x]
1    b    y  [b, y]
2    c    z  [c, z]


In [27]:
df['unido'][0]

['a', 'x']